In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter

import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from itertools import count

import gym

# set up matplotlib
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

In [2]:
device = torch.device("cpu")

# Gym Environment

In [3]:
import caviar_tools
from beamselect_env import BeamSelectionEnv

### Hyper Params

In [4]:
epi = [0,10] #[start,end]
gym_env = BeamSelectionEnv(epi)
n_steps = caviar_tools.linecount(epi)

/home/sundesh/Documents/git/ITU-Challenge/torch_rl/lib/python3.6/site-packages/gym/logger.py:34: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize("%s: %s" % ("WARN", msg % args), "yellow"))


In [5]:
n_steps

68700

Observation Space : X,Y,Z,pkts_dropped,pkts_transmitted,pkts_buffered,bit_rate

Action Space : [3,64] -> [UE,Possible beams]

In [6]:
gym_env.observation_space

Box([-500.  -500.  -500.    -1.    -1.    -1.5   -1.5    0.     0.     0.
    0. ], [5.0e+02 5.0e+02 5.0e+02 1.0e+00 1.0e+00 1.5e+00 1.5e+00 1.0e+03 1.0e+03
 2.0e+04 1.0e+09], (11,), float32)

In [7]:
state,reward,done,info=gym_env.step([2,53])

In [8]:
state

array(['11.417197', '37.027515', '7.4369965', '-0.0', '0.0', '0.9999752',
       '0.0070461035', '0.0', '22.0', '0.0', '1456463.7764599628'],
      dtype='<U32')

In [9]:
reward

0.0015139003578309937

In [10]:
info

{'pkts_dropped': 0.0,
 'pkts_transmitted': 22.0,
 'timestamp': '1626810037874447104',
 'obj': 'simulation_pedestrian1',
 'pos_x': '11.417197',
 'pos_y': '37.027515',
 'pos_z': '7.4369965',
 'orien_x': '-0.0',
 'orien_y': '0.0',
 'orien_z': '0.9999752',
 'orien_w': '0.0070461035',
 'linear_acc_x': '',
 'linear_acc_y': '',
 'linear_acc_z': '',
 'linear_vel_x': '',
 'linear_vel_y': '',
 'linear_vel_z': '',
 'angular_acc_x': '',
 'angular_acc_y': '',
 'angular_acc_z': '',
 'angular_vel_x': '',
 'angular_vel_y': '',
 'angular_vel_z': '',
 'pkts_buffered': 0.0,
 'bit_rate': 1456463.7764599628,
 'chosen_ue': 'simulation_pedestrian1',
 'packets': 14532.0,
 'channel_mag': array(0.00480021)}

## Replay Memory

In [11]:
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))


class ReplayMemory(object):

    def __init__(self, capacity):
        self.memory = deque([],maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

## DQN

In [12]:
class DQN(nn.Module):

    def __init__(self, inputs:int=7, outputs:int=64*3):
        super().__init__()

        self.linear = nn.Sequential(
            self.create_linear(inputs,16),
            self.create_linear(16, 32),
            self.create_linear(32,64),
            self.create_linear(64,outputs)
        )
    
    def create_linear(self,inp:int,out:int)-> nn.Module:
        return nn.Sequential(
            nn.Linear(inp,out),
            nn.ELU()
            # nn.BatchNorm1d(out)
        )
    
    def forward(self, x):
        x = x.to(device)
        x = self.linear(x)
        return x

        

In [13]:
tmp = DQN()
sample = torch.rand((1,7))
tmp(sample).shape

torch.Size([1, 192])

## Hyperparams

In [14]:
BATCH_SIZE = 128
GAMMA = 0.999
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 200
TARGET_UPDATE = 10

## Action

In [15]:
policy_net = DQN(11, 192).to(device)
target_net = DQN(11, 192).to(device)
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()

optimizer = optim.RMSprop(policy_net.parameters())
memory = ReplayMemory(10000)

n_actions = 64*3
steps_done = 0

def select_action(state):
    global steps_done
    sample = random.random()
    eps_threshold = EPS_END + (EPS_START - EPS_END) * \
        math.exp(-1. * steps_done / EPS_DECAY)
    steps_done += 1
    if sample > eps_threshold:
        with torch.no_grad():
            # t.max(1) will return largest column value of each row.
            # second column on max result is index of where max element was
            # found, so we pick action with the larger expected reward.
            flattened_action = policy_net(state).max(dim = 1).indices
            return torch.tensor([[flattened_action]], device=device, dtype=torch.long)
    else:
        flattened_action = random.randrange(n_actions)
        return torch.tensor([[flattened_action]], device=device, dtype=torch.long)


In [16]:
select_action(torch.tensor(state.astype(np.float32).reshape(1, state.shape[0])))

tensor([[87]])

## Optimize Model

In [17]:
def optimize_model():
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)
    # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
    # detailed explanation). This converts batch-array of Transitions
    # to Transition of batch-arrays.
    batch = Transition(*zip(*transitions))

    # Compute a mask of non-final states and concatenate the batch elements
    # (a final state would've been the one after which simulation ended)
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), device=device, dtype=torch.bool)
    non_final_next_states = torch.cat([s for s in batch.next_state
                                                if s is not None])
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)

    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken. These are the actions which would've been taken
    # for each batch state according to policy_net
    state_action_values = policy_net(state_batch).gather(1, action_batch)

    # Compute V(s_{t+1}) for all next states.
    # Expected values of actions for non_final_next_states are computed based
    # on the "older" target_net; selecting their best reward with max(1)[0].
    # This is merged based on the mask, such that we'll have either the expected
    # state value or 0 in case the state was final.
    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    next_state_values[non_final_mask] = target_net(non_final_next_states).max(1)[0].detach()
    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch
    # Compute Huber loss
    criterion = nn.SmoothL1Loss()
    loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))

    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    for param in policy_net.parameters():
        param.grad.data.clamp_(-1, 1)
    optimizer.step()

## Training loop

In [18]:
# Tensorboard
log_dir = './log_tensorboard/11_obs_space' 
writer = SummaryWriter(log_dir=log_dir)

In [19]:
# Initialize the environment and state
state = torch.zeros((1,11), dtype=torch.float32)
for i_episode in range(n_steps):
    # Select and perform an action
    action = select_action(state)
    # Observe new state
    next_state, reward, done, info = gym_env.step([action.item()//64, action.item()%64])
    next_state = next_state.astype(np.float32).reshape(1, next_state.shape[0])
    next_state = torch.tensor(next_state)
    reward = torch.tensor([reward], device=device).float()

    writer.add_scalar('episode_reward',reward,i_episode)

    # print(next_state)
    # print(state)
    # Store the transition in memory
    memory.push(state, action, next_state, reward)

    # Move to the next state
    state = next_state

    # Perform one step of the optimization (on the policy network)
    optimize_model()
    
    # Update the target network, copying all weights and biases in DQN
    if i_episode % TARGET_UPDATE == 0:
        target_net.load_state_dict(policy_net.state_dict())

print('Complete')
gym_env.close()

Complete
